<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Ознакомление-с-датасетом" data-toc-modified-id="Ознакомление-с-датасетом-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Ознакомление с датасетом</a></span></li><li><span><a href="#Подготовка-и-предобработка-данных" data-toc-modified-id="Подготовка-и-предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка и предобработка данных</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Деление-данных-на-обучающую-и-валидационную-выборки" data-toc-modified-id="Деление-данных-на-обучающую-и-валидационную-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Деление данных на обучающую и валидационную выборки</a></span></li><li><span><a href="#Обучение-модели-и-предсказания-на-валидационной-выборке" data-toc-modified-id="Обучение-модели-и-предсказания-на-валидационной-выборке-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели и предсказания на валидационной выборке</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Необходимо проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

### Ознакомление с датасетом

Импортируем библиотеки, которые нам понадобятся для дальнейшей работы

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

Откроем наши датасеты. Посмторим на на первые пять строк кажддого из них, также изучим общую информацию о них. 

In [2]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
for data in [df0, df1, df2]:
    display(data.head())
    data.info()
    print('----------------------------------------')

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
----------------------------------------


Согласно документации к данным:

* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей).

В результате первоначального ознакомления с датасетом, мы можем отметить, что каждый датасет состоит из 10 тыс. объектов и 5 признаков. Столбец `product` будет являться целевым признаком. Т. к. целевой признак количественный, то будет решаться задача регрессии.

Стоит отметить, признак `id` не будет влиять на обучение модели. В дальнейшей обработке данных, данный столбец будет удален. Также проверим наши данные на пропуски и дубликаты и посмотрим на основные статистические характеристики.

### Подготовка и предобработка данных

Удалим столбец `id`. Он не будут влиять на дальнейшее решение задачи.

In [4]:
def drop_id (data):
    del data['id']
    
for i in [df0, df1, df2]:
    drop_id(i)

Проверим наши данные на пропуски и дубликаты:

In [5]:
def isna_duplicate_check (data):
    print('Количество пропусков')
    print(data.isna().sum())
    print('Количество дубликатов')
    print(data.duplicated().sum())
    print('---------------------------------')

for i in [df0, df1, df2]:
    isna_duplicate_check(i)

Количество пропусков
f0         0
f1         0
f2         0
product    0
dtype: int64
Количество дубликатов
0
---------------------------------
Количество пропусков
f0         0
f1         0
f2         0
product    0
dtype: int64
Количество дубликатов
0
---------------------------------
Количество пропусков
f0         0
f1         0
f2         0
product    0
dtype: int64
Количество дубликатов
0
---------------------------------


Отлично! Дубликатов и пропусков не обнаружено

Далее посмотрим на статистические характеристики, дающие представление о выборке.

In [6]:
def data_describe (data):
    print(data.describe())
    print('--------------------------------------------------------------------')

for i in [df0, df1, df2]:
    data_describe(i)

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
--------------------------------------------------------------------
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%    

Данные изучены и подготовлены. Можно приступать к работе

## Обучение и проверка модели

### Деление данных на обучающую и валидационную выборки 

Разобьём наши данные на обучающую и валидационные выборки в соотношении 75:25.

In [7]:
target_0 = df0['product']
target_1 = df1['product']
target_2 = df2['product']
features_0 = df0.drop(['product'], axis=1)
features_1 = df1.drop(['product'], axis=1)
features_2 = df2.drop(['product'], axis=1)

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345
)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345
)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345
)

for i in [features_train_0, features_train_1, features_train_2]:
    print(i.shape)

(75000, 3)
(75000, 3)
(75000, 3)


### Обучение модели и предсказания на валидационной выборке

**Месторождение в 1-ом регионе**

In [8]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predictions_0 = model.predict(features_valid_0)
answers_0 = target_valid_0

print('Средний запас предсказанного сырья:', predictions_0.mean())
print('Средний запас фактического сырья:', answers_0.mean())
print('RMSE модели:', mean_squared_error(answers_0, predictions_0)** 0.5)

Средний запас предсказанного сырья: 92.59256778438035
Средний запас фактического сырья: 92.07859674082927
RMSE модели: 37.5794217150813


**Месторождение во 2-ом регионе**

In [9]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predictions_1 = model.predict(features_valid_1)
answers_1 = target_valid_1

print('Средний запас предсказанного сырья:', predictions_1.mean())
print('Средний запас фактического сырья:', answers_1.mean())
print('RMSE модели:', mean_squared_error(answers_1, predictions_1)** 0.5)

Средний запас предсказанного сырья: 68.728546895446
Средний запас фактического сырья: 68.72313602435997
RMSE модели: 0.893099286775617


**Месторождение в 3-ем регионе**

In [10]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predictions_2 = model.predict(features_valid_2)
answers_2 = target_valid_2

print('Средний запас предсказанного сырья:', predictions_2.mean())
print('Средний запас фактического сырья:', answers_2.mean())
print('RMSE модели:', mean_squared_error(answers_2, predictions_2)** 0.5)

Средний запас предсказанного сырья: 94.96504596800489
Средний запас фактического сырья: 94.88423280885438
RMSE модели: 40.02970873393434


Отсюда мы видим, что месторождение во втором регионе уступает по запасам сырья двум другим регионам, но по результатам метрки RMSE оно является самым выгодным и перспективным.

## Подготовка к расчёту прибыли

По условию залачи с помощью машинного обучения выбирают 200 лучших точек для разработки. Бюджет на разработку скважин в регионе - 10 млрд рублей. Посчитаем сколько придётся денег на разработку одной скважины, сохранив в переменной `budget_per_oil_well`.

In [11]:
development_budget = 10000000000 
budget_per_oil_well = development_budget / 200
print('Бюджет на разработку одной скважины:', budget_per_oil_well)

Бюджет на разработку одной скважины: 50000000.0


Также по условию сказано, что доход с каждой единицы продукта составляет 450 тыс. рублей. Далее посчитаем объём сырья, который необходим, чтобы окупить затраты, и выведем полученный результат на экран.

In [12]:
revenue_per_unit_of_product = 450000
breakeven_point = budget_per_oil_well / revenue_per_unit_of_product   #точка безубыточности

print('Достаточный объём сырья для безубыточной разаработки новой скважины:', breakeven_point)

Достаточный объём сырья для безубыточной разаработки новой скважины: 111.11111111111111


Сравним полученный объем сырья со средним запасом в каждом регионе

In [13]:
print('Объём сырья для безубыточной разработки новой скважины 1 месторождения:', answers_0.mean() - breakeven_point)
print('Объём сырья для безубыточной разработки новой скважины 2 месторождения:', answers_1.mean() - breakeven_point)
print('Объём сырья для безубыточной разработки новой скважины 3 месторождения:', answers_2.mean() - breakeven_point)

Объём сырья для безубыточной разработки новой скважины 1 месторождения: -19.032514370281845
Объём сырья для безубыточной разработки новой скважины 2 месторождения: -42.38797508675114
Объём сырья для безубыточной разработки новой скважины 3 месторождения: -16.226878302256736


По результатам наших расчётов мы видим, что самая затратная скважина будет находиться во 2-ом регионе. Необходимо больше времени для того, чтобы окупить вложеннный бюджет для разработки. Для скважин в 1-ом и 3-ем регионах срок окупаемости в 2 раза меньше.

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам (топ-200) и предсказаниям модели:

In [14]:
def revenue(target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    predictions_sorted = predictions.sort_values(ascending=False)[:200].index
    selected = target[predictions_sorted]
    revenue = selected.sum() * revenue_per_unit_of_product - development_budget
    return (revenue / 1000000).round(2)

Посчитаем прибыль для каждого региона:

In [15]:
revenue_0 = revenue(answers_0, predictions_0)
revenue_1 = revenue(answers_1, predictions_1)
revenue_2 = revenue(answers_2, predictions_2)
print('Средняя прогнозная прибыль в 1-ом регионе:', revenue_0, 'млн руб.')
print('Средняя прогнозная прибыль во 2-ом регионе:', revenue_1, 'млн руб.')
print('Средняя прогнозная прибыль в 3-ем регионе:', revenue_2, 'млн руб.')

Средняя прогнозная прибыль в 1-ом регионе: 3320.83 млн руб.
Средняя прогнозная прибыль во 2-ом регионе: 2415.09 млн руб.
Средняя прогнозная прибыль в 3-ем регионе: 2710.35 млн руб.


Далее при помощи техники Bootstrap найдём среднюю прибыль, 95-% доверительный интервал и риск убытков.

In [16]:
def bootstrap(target, predictions, iterations, sample_size):
    state = np.random.RandomState(12345)
    values = []
    for i in range(iterations):
        target_subsample = target.sample(n=sample_size, random_state=state, replace=True)
        pred_subsample = predictions[target_subsample.index]
        values.append(revenue(target_subsample, pred_subsample))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    quantile_interval = (lower, upper)
    return round(mean,2), confidence_interval, (risk*100).round(2), quantile_interval

In [17]:
mean_0, interval_0, risk_0, quantile_0 = bootstrap(answers_0.reset_index(drop=True), predictions_0, 1000, 500)
print(f'Средняя прибыль в 1-ом регионе: {mean_0}')
print(f'95 %ый доверительный интервал по стьюденту в 1-ом регионе: {interval_0}')
print(f'95 %ый доверительный интервал по квантилям в 1-ом регионе: {quantile_0}')
print(f'Риск убытков в 1-ом регионе: {risk_0} %')

Средняя прибыль в 1-ом регионе: 396.17
95 %ый доверительный интервал по стьюденту в 1-ом регионе: (379.62035601496876, 412.70970398503124)
95 %ый доверительный интервал по квантилям в 1-ом регионе: (-111.21425, 909.7642499999998)
Риск убытков в 1-ом регионе: 6.9 %


In [18]:
mean_1, interval_1, risk_1, quantile_1 = bootstrap(answers_1.reset_index(drop=True), predictions_1, 1000, 500)
print(f'Средняя прибыль во 2-ом регионе: {mean_1}')
print(f'95 %ый доверительный интервал по стьюденту во 2-ом регионе: {interval_1}')
print(f'95 %ый доверительный интервал по квантилям во 2-ом регионе: {quantile_1}')
print(f'Риск убытков во 2-ом регионе: {risk_1} %')

Средняя прибыль во 2-ом регионе: 456.05
95 %ый доверительный интервал по стьюденту во 2-ом регионе: (443.1472043267659, 468.9429356732341)
95 %ый доверительный интервал по квантилям во 2-ом регионе: (33.82000000000001, 852.28575)
Риск убытков во 2-ом регионе: 1.5 %


In [19]:
mean_2, interval_2, risk_2, quantile_2 = bootstrap(answers_2.reset_index(drop=True), predictions_2, 1000, 500)
print(f'Средняя прибыль в 3-ем регионе: {mean_2}')
print(f'95 %ый доверительный интервал по стьюденту в 3-ем регионе: {interval_2}')
print(f'95 %ый доверительный интервал по квантилям в 3-ем регионе: {quantile_2}')
print(f'Риск убытков в 3-ем регионе: {risk_2} %')

Средняя прибыль в 3-ем регионе: 404.4
95 %ый доверительный интервал по стьюденту в 3-ем регионе: (387.44576760085766, 421.3618923991423)
95 %ый доверительный интервал по квантилям в 3-ем регионе: (-163.34949999999998, 950.36325)
Риск убытков в 3-ем регионе: 7.6 %


## Вывод

По резульататам исследования, рекомендовано произвести разработку скважин во 2-ом регионе. Данный выбор обоснован тем, что в данном регионе отмечается высокий уровень средней прибыли. Также стоит отметить, что риски убытков в данном регионе минимальные из трёх возможных.